In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.version as ver
from biogeme import models
from biogeme.expressions import Beta, log

Version of Biogeme

In [2]:
print(ver.getText())

biogeme 3.2.8 [2021-09-02]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



Results of the experiment from Ariely (2008):

|     | Experiment 1            | Experiment 2           |     |
| --- | ---                     | ---                    | --- |
|16   | Web only @ \$59         | Web only @ \$59        | 68  |
|0    | Print only @ \$125      |                        |     |
|84   | Print and web @ \$125   | Print and web @ \$125  | 32  |

In [3]:
data = pd.DataFrame({'Experiment': [1, 2], 
                     'Web_only': [16, 68], 
                     'Print_and_web': [84, 32]})


In [4]:
data

,Experiment,Web_only,Print_and_web
0,1,16,84
1,2,68,32


In [5]:
database = db.Database('data', data)
globals().update(database.variables)

# Logit model

## Parameters 

We estimate two parameters:
- a coefficient for the cost,
- a constant associated with alternatives involving the web option.

In [6]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
cte_web = Beta('cte_web', 0, None, None, 0)

## Utility functions

In [7]:
V_web = beta_cost * 59 + cte_web
V_print = beta_cost * 125
V_print_web = beta_cost * 125 + cte_web

In [8]:
V = {
    1: V_web, 
    2: V_print, 
    3: V_print_web
}

## Availability conditions

Alternative 2 (print only) is only available in experiment 1.

In [9]:
av = {1: 1, 2: Experiment == 1, 3: 1}

## Log likelihood

As alternative 2 is never chosen, it does not appear in the log likelihood function.

In [10]:
loglike = (
    Web_only * models.loglogit(V, av, 1) + 
    Print_and_web * models.loglogit(V, av, 3)
)

In [11]:
biogeme = bio.BIOGEME(database, loglike)
biogeme.modelName = 'logit'

In [12]:
results = biogeme.estimate()

In [13]:
print(results.printGeneralStatistics())

Number of estimated parameters:	2
Sample size:	2
Excluded observations:	0
Init log likelihood:	-136.0584
Final log likelihood:	-136.0584
Likelihood ratio test for the init. model:	4.331306e-05
Rho-square for the init. model:	1.59e-07
Rho-square-bar for the init. model:	-0.0147
Akaike Information Criterion:	276.1168
Bayesian Information Criterion:	273.5031
Final gradient norm:	3.4960E-04
Nbr of threads:	16



In [14]:
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,0.004891,0.002171,2.252949,0.024262,0.011435,0.427678,0.668886
cte_web,15.341971,281.677898,0.054466,0.956564,1.244489,12.327932,0.000000


We note that the cost coefficient is not significantly different from zero.

# Logit model with a decoy

We introduce now a variable that identifies the presence of the decoy. The behavioral assumption is that the "print and web" alternative becomes attractive because customers perceive that they obtain a significant discount, as they can obtain this alternative for the same price as "print only". Therefore, the presence of the dominated alternative "print only" (in experiment 1) increases the perceived utility of the alternative "print and web".

In [15]:
decoy = Experiment == 1
beta_decoy = Beta('beta_decoy', 0, None, None, 0)

In [16]:
V_print_web_decoy = (
    beta_cost * 125 +  
    cte_web +
    beta_decoy * decoy
)

In [17]:
V_decoy = {
    1: V_web, 
    2: V_print, 
    3: V_print_web_decoy
}

In [18]:
loglike_decoy = (
    Web_only * models.loglogit(V_decoy, av, 1) + 
    Print_and_web * models.loglogit(V_decoy, av, 3)
)

In [19]:
biogeme_decoy = bio.BIOGEME(database, loglike_decoy)
biogeme.modelName = 'logit_decoy'

In [20]:
results_decoy = biogeme_decoy.estimate()

In [21]:
print(results_decoy.printGeneralStatistics())

Number of estimated parameters:	3
Sample size:	2
Excluded observations:	0
Init log likelihood:	-106.654
Final log likelihood:	-106.654
Likelihood ratio test for the init. model:	8.282283e-05
Rho-square for the init. model:	3.88e-07
Rho-square-bar for the init. model:	-0.0281
Akaike Information Criterion:	219.3079
Bayesian Information Criterion:	215.3874
Final gradient norm:	2.5649E-04
Nbr of threads:	16



In [22]:
results_decoy.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.011421,0.003248,-3.516166,4.378276e-04,2.753867e-13,-4.147181e+10,0.0
beta_decoy,2.412000,0.346930,6.952404,3.591127e-12,3.385965e-11,7.123522e+10,0.0
cte_web,12.652095,203.698434,0.062112,9.504737e-01,1.000000e+00,1.265209e+01,0.0


Now, the cost coefficient is negative, and significantly different from zero. And the impact of the decoy is to increase the utility of the "print and web" alternative, as expected. 

In this example, we have been able to capture the apparent irrationality reported by Ariely (2008) with a simple specification of the utility functions, resulting from the behavioral assumption that customers are attracted by alternatives associated with a discount. More complex examples may involve subjectivity, perception, and attitudes to address the apparent irrationality. These behavioral concepts cannot easily be captured by observed variables. But, as we will see later, they can be captured by so-called "latent variables", that can be included in the choice models. 